# Question-Answering Model

## Importing the libraries

In [33]:
from exchangelib import Credentials, Account
from datetime import datetime
import pytz
from transformers import pipeline
import torch
import numpy as np
import pandas as pd
import json
from pickle import dump

## EWS API to access Exchange Server Mailboxes

In [28]:
#Authenticate with Email and Password:
email = 'ingrid.test1@outlook.com'
password = 'Maslow94'

credentials = Credentials(email, password)
account = Account(email, credentials=credentials, autodiscover=True)

#Define Variables

In [29]:
# Define the queries
questions = [
    "What is the vendor's name",
    "What is the client's name",
    "What is the payment rate"
]
#Define Variables
email_answers_list = []
answers_dict = {}
tz = pytz.timezone('America/New_York')  # timezone


# Create a Pipeline for Question-Answering 

In [32]:
# # Create a pipeline for question answering
# qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad", tokenizer="bert-large-uncased")

qa_pipeline = pipeline(
    "question-answering",
    model="bert-large-uncased-whole-word-masking-finetuned-squad",
    tokenizer="bert-large-uncased",
    framework="pt"  # Explicitly specify PyTorch
)

# Fetch Emails from a Date Range and feed the LLM

In [ ]:
# Define the start and end date with timezone information
#start_date = tz.localize(datetime(2023, 9, 21, 0, 0, 0))
#end_date = tz.localize(datetime(2023, 9, 21, 23, 59, 59))
####################

# Get the current date in the Atlanta time zone
current_date_atlanta = datetime.now(tz).date()
# Define the start and end date to cover today in the Atlanta time zone
start_date = tz.localize(datetime(current_date_atlanta.year, current_date_atlanta.month, current_date_atlanta.day, 0, 0, 0))
end_date = tz.localize(datetime(current_date_atlanta.year, current_date_atlanta.month, current_date_atlanta.day, 23, 59, 59))
###################

# Filter Emails based on the start and end dates
emails = account.inbox.filter(datetime_received__range=(start_date, end_date))

In [ ]:
#Iterate thorugh the emails' content to feed the LLM and get the answers to the defined queries in return

if not emails.count():
    print("No emails found in the QuerySet.")
else:
    # Iterate through the QuerySet and print the type of each item
    for email in emails:
        print(f"Type of item: {type(email)}")
        
        ############################################################
        # Get the Subject and Sender of the email
        print(f"Subject: {email.subject}")
        print(f"Sender: {email.sender.email_address}")
        
        # Get the plain text content of the email
        email_content=email.text_body

        if email_content:
            print(f"Plain Text Content:\n{email_content}")
            answers_dict = {}

            # Process each question and store the answers
            for question in questions:
                result = qa_pipeline(question=question, context=email_content)
                answer = result["answer"]
                answers_dict[question] = answer
            
            email_answers_list.append(answers_dict)

# FROM JSON TO DF

In [ ]:
# Convert the list of dictionaries into a JSON structure
json_data = json.dumps(email_answers_list, indent=4)

# Convert the Jason Structure into a DataFrame
df = pd.read_json(json_data)

# Rename the columns of the dataframe
df.columns = ["Vendor", "Client", "Payment Rate"]
df.index=df.index+1

In [ ]:
#Visualize the dataframe
df

# Outlook Function

In [ ]:
def outlook_function (user, password, start_date, end_date):
    #ALL THE above CODE
    return csv_file(df)


# Pickle the Function with the model

In [ ]:
# save the model
#dump(outlook_function, open('model.pkl', 'wb'))